In [ ]:
import pyspark
from pyspark.sql.context import SQLContext
from pyspark.sql.types import *
sc = pyspark.SparkContext('local[*]') 

## Ejercicios propuestos

**Ejercicio 1**: Escribir una UDF que convierta en minúsculas excepto el primer caracter de la columna *des_nomco* del dataframe *ttgoficiDF* cargado a continuación: 

In [ ]:
sqlContext = SQLContext(sc)
dataPath = "../data/ttgofici.csv"
ttgoficiDF = sqlContext.read.format("com.databricks.spark.csv")\
            .option("header", "true")\
            .load(dataPath) 

In [ ]:
ttgoficiDF.show(2)

In [ ]:
# x.capitalize()
def lower(string):
    return string[0] + string[1:].lower()

In [ ]:
def lower2(string):
    return string.capitalize()

In [ ]:
from pyspark.sql.functions import col,udf
udLower = udf(lower, StringType())
ttgoficiDF.withColumn("des_nomab_lower",udLower("des_nomab")).show()

**Ejercicio 2**: Desarrollar una "UDF" que convierta el primer caracter de un string según la siguiente regla:    
'1'-> 'a'  
'2'-> '1'  
'3'-> '2'  
'4'-> '3'  
'5'-> '4'  
'6'-> '5'  
'7'-> '6'  
'8'-> '7'  
De tal forma un String con valor: "1.Semana del 1 al 7 de agosto" se convertirá a : "a..Semana del 1 al 7 de   agosto", "2.Semana del 8 al 14 de agosto" se convertirá a "1.Semana del 8 al 14 de agosto"... y así sucesivamente.  
Tip: utilizar el siguiente mapping:  
mapping = {'1': 'a', '2': '1', '3': '2', '4': '3', '5': '4', '6': '5', '7': '6', '8':'7'}

In [ ]:
data = [('1', '1.Semana del 1 al 7 de agosto'),('2', '2.Semana del 8 al 14 de agosto'),\
     ('3', '3.Semana del 15 al 21 de agosto'),('4', '3.Semana del 15 al 21 de agosto')]
DFSpentData =  sqlContext.createDataFrame(data, ['id','Semana'])

In [ ]:
DFSpentData.show()

In [ ]:
mapping = {'1': 'a', '2': '1', '3': '2', '4': '3', '5': '4', '6': '5', '7': '6', '8':'7'}

In [ ]:
from pyspark.sql.functions import UserDefinedFunction
udfSemana = UserDefinedFunction(lambda x: mapping[x[0]] + x[1:], StringType())

In [ ]:
DFSpentData.withColumn("semana2",udfSemana("semana")).collect()

### In sql sentence...

In [ ]:
sqlContext.registerDataFrameAsTable(DFSpentData,"SpentData")

In [ ]:
sqlContext.registerFunction("getDay", lambda x: mapping[x[0]] + x[1:], StringType())

In [ ]:
sqlContext.sql("select id, udfSemana(Semana) as semana3 from SpentData").collect()

**Ejercicio 3**: Migrar las siguientes transformaciones en lenguaje a SAS a SparkSQL:

data inframul_diaria;  
	set diaria.Nuevanet_di_con;  

	dia = substr(FEC_ULTACCSO,9,2);  
	mes = substr(FEC_ULTACCSO,6,2);  
	anio= substr(FEC_ULTACCSO,1,4);  
	fecha = compress(anio||mes||dia);  
	format fecha_ult_conexion_semanal date9.;  
	fecha_ult_conexion_semanal =input(fecha, yymmdd10.);  
	cod_persona = input (COD_PERSCTPN, 11.);  
run;

proc sql;  
	create table cli_net_semanal as  
	select distinct cod_persona,  
			max (fecha_ult_conexion_semanal) as fecha_ult_conexion_semanal format date9.  
	from inframul_diaria  
	where fecha_ult_conexion_semanal>=&FEC_INI02  
	group by cod_persona 
	order by cod_persona;  
quit;


In [ ]:
query = "Select *, cod_persctpn as cod_persona FROM nuevanet_di_con"
udf_dia = UserDefinedFunction(lambda x: x[8:10], StringType())
udf_mes = UserDefinedFunction(lambda x: x[5:7], StringType())
udf_anio = UserDefinedFunction(lambda x: x[0:4], StringType())
udf_fecha = UserDefinedFunction(lambda x,y,z: x+y+z, StringType())
udf_fecha_mes = UserDefinedFunction(lambda x,y,z: x+mes_l[y]+z, StringType())
campo_fecha = 'fec_ultaccso'

sqlContext.sql(query).withColumn("dia",udf_dia(campo_fecha))\
.withColumn("mes",udf_mes(campo_fecha))\
.withColumn("anio",udf_anio(campo_fecha))\
.withColumn("fecha",udf_fecha("dia","mes","anio"))\
.withColumn("fecha_ult_conexion_semanal",udf_fecha_mes("dia","mes","anio"))\
.registerTempTable("inframul_diaria")

In [ ]:
FEC_INI02 = "01ENE2000"
sqlContext.sql("Select distinct cod_persona, max(fecha_ult_conexion_semanal) as fecha_ult_conexion_semanal \
FROM inframul_diaria \
WHERE  fecha_ult_conexion_semanal>='" + FEC_INI02 + "'\
GROUP BY cod_persona")\
.registerTempTable("cli_net_semanal")